In [2]:
# From https://kioku-space.com/en/jupyter-skip-execution/, a method of skipping unwanted executions in ipynb cells
from IPython.core.magic import register_cell_magic

@register_cell_magic
def skip(line, cell):
    return

In [3]:
import networkx as nx
import numpy as np
import pyvista as pv
import tetgen
from scipy.optimize import minimize, least_squares
from scipy.spatial.transform import Rotation as R
import open3d as o3d
import time
import pickle
import base64
import stl
from stl import mesh
import os

from lib3mf_common import *

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
# Load mesh
model_name = "pi 3mm"
model_path = f'input_models/{model_name}.stl'
model_mesh = mesh.Mesh.from_file(model_path)

# save mesh as binary stl
model_mesh.save(f'input_models/{model_name}_binary.stl')
# save mesh as ASCII stl
model_mesh.save(f'input_models/{model_name}_ascii.stl', mode=stl.Mode.ASCII)

# read triangle mesh
if os.path.isfile(f'input_models/{model_name}.stl'):
    print("Found file!")
    model_mesh = o3d.io.read_triangle_mesh(f'input_models/{model_name}.stl')

# convert to tetrahedral mesh
input_tet = tetgen.TetGen(np.asarray(model_mesh.vertices), np.asarray(model_mesh.triangles))
# input_tet.make_manifold() # comment out if not needed
input_tet.tetrahedralize()
input_tet = input_tet.grid

Found file!


In [5]:
# TEST CODE BLOCK
# extract vertices from 3mf via lib3mf, from tet surface via pyvista
# edit 3mf vertices
PART_OFFSET = np.array([0., 0., 0.])
x_min, x_max, y_min, y_max, z_min, z_max = input_tet.bounds
bounding_box = np.array([(x_min + x_max) / 2, (y_min + y_max) / 2, z_min]) + PART_OFFSET
input_tet.points -= bounding_box

def shift_vert_to_center(lib3mf_Vertex):
    x, y, z = lib3mf_Vertex.Coordinates
    x = x - bounding_box[0]
    y = y - bounding_box[1]
    z = z - bounding_box[2]
    new_vert = lib3mf.Position()
    new_vert.Coordinates[0] = x
    new_vert.Coordinates[1] = y
    new_vert.Coordinates[2] = z
    return new_vert

# getting vertices from lib3mf
model_path = "./input_models/pi 3mm.3mf"
if os.path.isfile(model_path):
    wrapper = get_wrapper()
    wrapper = wrapper.CreateModel()
    read_3mf_file_to_model(wrapper, model_path)
    print(type(wrapper))
    print(type(wrapper.GetMeshObjectByID(1)))
    print(type(wrapper.GetMeshObjectByID(1).GetVertexCount()))
    print(wrapper.GetMeshObjectByID(1).GetVertexCount())
    for i in range(5):
        # print(wrapper.GetMeshObjectByID(1).GetVertex(i))
        # print(type(wrapper.GetMeshObjectByID(1).GetVertex(i)))
        # print(wrapper.GetMeshObjectByID(1).GetVertex(i)) 
        # get vertex returns an sPosition struct with 3 m_Coordinates
        print([x for x in wrapper.GetMeshObjectByID(1).GetVertex(i).Coordinates])
        # okay so the above will return the vertices and yes they are in order
    # doing a deformation over all vertices (subtract bounding box)
    for i in range(5):
        new_vect = shift_vert_to_center(wrapper.GetMeshObjectByID(1).GetVertex(i))
        wrapper.GetMeshObjectByID(1).SetVertex(i, new_vect)
        print([x for x in wrapper.GetMeshObjectByID(1).GetVertex(i).Coordinates])

surface = input_tet.extract_surface() #pyvista
points = input_tet.points
print(surface.get_cell(0).points)
print(surface.number_of_cells)
print(surface.number_of_points) # points are vertices, cells are triangles. Hoping 1:1 correspondence
print("Unsorted Points:")
for i in range(5):
    print(points[i])
# TEST BLOCK SUCCESSFUL

<class 'lib3mf.Lib3MF.Model'>
<class 'lib3mf.Lib3MF.MeshObject'>
<class 'int'>
437
[40.0, 40.0, 10.0]
[37.5, 40.0, 10.0]
[40.0, 42.5, 10.0]
[37.5, 42.5, 10.0]
[40.0, 45.0, 10.0]
[25.0, -5.0, 10.0]
[22.5, -5.0, 10.0]
[25.0, -2.5, 10.0]
[22.5, -2.5, 10.0]
[25.0, 0.0, 10.0]
[[25.  -2.5 10. ]
 [25.   0.  12.5]
 [25.  -2.5 12.5]]
870
437
Unsorted Points:
[25.  -2.5 10. ]
[22.5 -2.5 10. ]
[25.   2.5 10. ]
[25.  5. 10.]
[22.5  2.5 10. ]


In [6]:
# TEST CODE BLOCK
# edit all 3mf vertices
# establish a correspondence list with the vertices of tet surface

bounding_box = np.array([(x_min + x_max) / 2, (y_min + y_max) / 2, z_min])

def shift_vert_to_center(lib3mf_Vertex):
    x, y, z = lib3mf_Vertex.Coordinates
    x = x - bounding_box[0]
    y = y - bounding_box[1]
    z = z - bounding_box[2]
    new_vert = lib3mf.Position()
    new_vert.Coordinates[0] = x
    new_vert.Coordinates[1] = y
    new_vert.Coordinates[2] = z
    return new_vert

def mf_verts_to_np_array(lib3mf_Vertex):
    x, y, z = lib3mf_Vertex.Coordinates
    return np.array([x, y, z])

mf_num_verts = 0
# getting vertices from lib3mf
model_path = "./input_models/pi 3mm.3mf"
if os.path.isfile(model_path):
    wrapper = get_wrapper()
    wrapper = wrapper.CreateModel()
    read_3mf_file_to_model(wrapper, model_path)
    mesh_obj = wrapper.GetMeshObjectByID(1)
    # doing a deformation over all vertices (subtract bounding box)
    mf_num_verts = mesh_obj.GetVertexCount()
    for i in range(mf_num_verts):
        new_vect = shift_vert_to_center(mesh_obj.GetVertex(i))
        mesh_obj.SetVertex(i, new_vect)

surface = input_tet.extract_surface() #pyvista
# tet_verts = input_tet.points # number of points in tetmesh do NOT = number points in surface STL or 3mf
surf_verts = surface.points
tet_num_verts = surface.number_of_points

# for i in range(10): # still would be nice if no matching to do
#     print(f"tet surface vertex {i}: {surf_verts[i]}")
#     print(f"3mf vertex {i}: {[x for x in mesh_obj.GetVertex(i).Coordinates]}")

vert_correspondence = np.full(tet_num_verts, -1)
have_correspondence = True
# check if number of vertices in 3mf and tet extracted surface match
if (mf_num_verts == tet_num_verts):
    # if they do, iterate over 3mf vertices because easier to compare with list in tet vert
    print("Surface vertices from tetmesh are:\n", surf_verts)
    print("Vertices from mesh_obj are", [[x.Coordinates[0], x.Coordinates[1], x.Coordinates[2]] for x in mesh_obj.GetVertices()])
    for i in range(100):
        # get current vertex
        curr_vert = mesh_obj.GetVertex(i)
        curr_vert = mf_verts_to_np_array(curr_vert)

        # find the index of vertex in tetgen where they match
        # surf_verts is a 2D list, surf_verts[i] returns a 3-length list with one vertex
        
        # print("matching with lists")
        # my_list = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
        # index = my_list.index([4, 5, 6])
        # print(index)

        # print("matching with np arrays")
        # my_list = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
        # matcher = np.array([4, 5, 6])
        # print(my_list.shape)
        # index, j = np.where((my_list < 0), my_list, my_list*10)
        # print(index)
        # print(j)

        # a = np.array([1, 2, 3])
        # b = np.array([1, 2, 3])
        # print(a == b)
        # print((a==b).all())

        # a = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
        # b = np.array([7, 8, 9])
        # # condition, x, y ; selects from x when true else y
        # # so I don't think this is what we want; x would be need to be indices, y would be -1, can't rly compare a
        # print(np.where((a==b))) # odd -- why returns [2, 2, 2]? they ARE same in index 2 but broadcast to all
        # # np.where is confusing; myb diff way to do this
        # print(np.asarray(a==b).nonzero()) # these two equivalent -- this returns indices where the condition is nonzero
        # print(np.isin(a, b))
        # mask = [np.isin(a, b)[x].all() for x in range(np.isin(a, b).shape[1])]
        # print([np.isin(a, b)[x].all() for x in range(np.isin(a, b).shape[1])])
        # print(a[mask])
        # print(np.where(mask))

        # okay, so
        # mask surf_verts with true wherever surf_verts = curr_vert, false elsewhere
        print("Current vertex is: ", curr_vert)
        match_vert_mask = np.isin(surf_verts, curr_vert) 
        # filter for partial matches by using np.all
        mask = np.array([match_vert_mask[x].all() for x in range(match_vert_mask.shape[1])])
        # print indices where mask is true
        print("Match between surf and curr 3mf vert at this index of surf_verts:", np.where(mask))
        # ideally there would be only one
   
         

        # print(curr_vert)
        # print(surf_verts[0])
        
        # match_index = np.where(np.isclose(surf_verts, curr_vert))
        # print(match_index)

        # check that there is a match, else correspondence is false

        # check that the entry in correspondence list is -1, else correspondence is false (doubly-def vertex means we dropped one)

    # if any elements in the list still = -1, then have unmatched vertices and correspondence fails
    
    
    print(list(surf_verts[surf_verts[:, 1].argsort()]))

else:
    print("Number of Vertices do not match!")

if(have_correspondence):
    print("Correspondence Successful!")
# TEST BLOCK BROKEN

Surface vertices from tetmesh are:
 [[25.         -2.5        10.        ]
 [25.          0.         12.5       ]
 [25.         -2.5        12.5       ]
 ...
 [-7.35294104  5.         17.1428566 ]
 [-4.4117651   5.         17.1428566 ]
 [-1.47058868  5.         17.1428566 ]]
Vertices from mesh_obj are [[25.0, -5.0, 10.0], [22.5, -5.0, 10.0], [25.0, -2.5, 10.0], [22.5, -2.5, 10.0], [25.0, 0.0, 10.0], [22.5, 0.0, 10.0], [25.0, 2.5, 10.0], [22.5, 2.5, 10.0], [25.0, 5.0, 10.0], [22.5, 5.0, 10.0], [20.0, 2.5, 10.0], [20.0, 5.0, 10.0], [17.5, 2.5, 10.0], [17.5, 5.0, 10.0], [15.0, 2.5, 10.0], [15.0, 5.0, 10.0], [20.0, -5.0, 10.0], [20.0, -2.5, 10.0], [20.0, 0.0, 10.0], [17.5, -5.0, 10.0], [17.5, -2.5, 10.0], [17.5, 0.0, 10.0], [15.0, -5.0, 10.0], [15.0, -2.5, 10.0], [15.0, 0.0, 10.0], [25.0, -5.0, 20.0], [25.0, -5.0, 17.5], [25.0, -2.5, 20.0], [25.0, -2.5, 17.5], [25.0, 0.0, 20.0], [25.0, 0.0, 17.5], [25.0, 2.5, 20.0], [25.0, 2.5, 17.5], [25.0, 5.0, 20.0], [25.0, 5.0, 17.5], [25.0, 2.5, 15.0]

In [7]:
# TEST CODE BLOCK
# doing the above but right after tetrahedralize

# Load mesh
model_name = "pi 3mm"
model_path = f'input_models/{model_name}.stl'
# make binary stl from ASCII
model_mesh = mesh.Mesh.from_file(model_path)
model_mesh.save(f'input_models/{model_name}_binary.stl')
# make ASCII stl from binary
model_mesh.save(f'input_models/{model_name}_ascii.stl', mode=stl.Mode.ASCII)
# debug statements
# read triangle mesh
if os.path.isfile(f'input_models/{model_name}.stl'):
    print("Found file!")
    model_mesh = o3d.io.read_triangle_mesh(f'input_models/{model_name}_ascii.stl')

# convert to tetrahedral mesh
input_tet = tetgen.TetGen(np.asarray(model_mesh.vertices), np.asarray(model_mesh.triangles))
# input_tet.make_manifold() # comment out if not needed
input_tet.tetrahedralize()
input_tet = input_tet.grid

def mf_verts_to_np_array(lib3mf_Vertex):
    x, y, z = lib3mf_Vertex.Coordinates
    return np.asarray([[x, y, z]])

mf_num_verts = 0
# getting vertices from lib3mf
model_path = "./input_models/pi 3mm.3mf"
if os.path.isfile(model_path):
    wrapper = get_wrapper()
    wrapper = wrapper.CreateModel()
    read_3mf_file_to_model(wrapper, model_path)
    mesh_obj = wrapper.GetMeshObjectByID(1)
    # doing a deformation over all vertices (subtract bounding box)
    mf_num_verts = mesh_obj.GetVertexCount()

surface = input_tet.extract_surface() #pyvista
# surf_verts = [list(model_mesh.vertices[i]) for i in range(len(model_mesh.vertices))]
# surf_verts = input_tet.points # number of points in tetmesh do NOT = number points in surface STL or 3mf
surf_verts = surface.points
tet_verts = input_tet.points
# mesh_verts = [list(model_mesh.vertices[i]) for i in range(len(model_mesh.vertices))]
mesh_verts = np.asarray(model_mesh.vertices)
mf_verts = np.asarray([[x.Coordinates[0], x.Coordinates[1], x.Coordinates[2]] for x in mesh_obj.GetVertices()])

vert_correspondence = np.full(mf_num_verts, -1)
have_correspondence = True

# if they do, iterate over 3mf vertices because easier to compare with list in tet vert
print("Surf vertices are:\n", surf_verts)
print("Tet vertices are:\n", tet_verts)
print("Mesh verts are: \n", mesh_verts)
print("Vertices from 3mf are:\n", mf_verts)


correspondence_list = []
for i in range(2):
    # get current vertex
    curr_vert = mesh_obj.GetVertex(i)
    curr_vert = mf_verts_to_np_array(curr_vert)

    # okay, so
    # mask surf_verts with true wherever surf_verts = curr_vert, false elsewhere
    # print("Current vertex is: ", curr_vert)
    match_vert_mask = (surf_verts[:, :] == curr_vert[:, None]).all(axis=-1).any(axis=0) 
    print("curr vert:\n", curr_vert)
    print("mesh_verts:\n", [mesh_verts[j] for j in range(3)])
    print(match_vert_mask)
    # # filter for partial matches by using np.all
    # mask = np.array([match_vert_mask[x].all() for x in range(match_vert_mask.shape[1])])
    # # print indices where mask is true
    # where_true = np.where(mask)
    # correspondence_list.append(list(where_true[0]))
    # # print("Match between tet points and curr 3mf vert at this index of surf_verts:", np.where(mask))
    # # ideally there would be only one
print("Correspondences are:\n", correspondence_list)
# TEST BLOCK BROKEN

Found file!
Surf vertices are:
 [[40.         42.5        10.        ]
 [40.         45.         12.5       ]
 [40.         42.5        12.5       ]
 ...
 [ 7.64705896 50.         17.1428566 ]
 [10.5882349  50.         17.1428566 ]
 [13.52941132 50.         17.1428566 ]]
Tet vertices are:
 [[40.         42.5        10.        ]
 [37.5        42.5        10.        ]
 [40.         47.5        10.        ]
 ...
 [12.14642477 45.         15.6175632 ]
 [34.64822317 45.         14.81111126]
 [19.15356502 45.         15.32433629]]
Mesh verts are: 
 [[40.         40.         10.        ]
 [37.5        40.         10.        ]
 [40.         42.5        10.        ]
 ...
 [-7.05882311 50.         17.1428566 ]
 [-4.11764717 50.         14.28571415]
 [-7.05882311 50.         14.28571415]]
Vertices from 3mf are:
 [[40.         40.         10.        ]
 [37.5        40.         10.        ]
 [40.         42.5        10.        ]
 ...
 [ 1.76470494 50.         14.28571415]
 [-1.17647004 50.         

In [8]:
%%skip
bounding_box = np.array([(x_min + x_max) / 2, (y_min + y_max) / 2, z_min])

def shift_vert_to_center(lib3mf_Vertex):
    x, y, z = lib3mf_Vertex.Coordinates
    x = x - bounding_box[0]
    y = y - bounding_box[1]
    z = z - bounding_box[2]
    new_vert = lib3mf.Position()
    new_vert.Coordinates[0] = x
    new_vert.Coordinates[1] = y
    new_vert.Coordinates[2] = z
    return new_vert

def mf_verts_to_np_array(lib3mf_Vertex):
    x, y, z = lib3mf_Vertex.Coordinates
    return np.array([x, y, z])

# load 3mf
mf_num_verts = 0
# getting vertices from lib3mf
model_path = "./input_models/pi 3mm.3mf"
if os.path.isfile(model_path):
    wrapper = get_wrapper()
    wrapper = wrapper.CreateModel()
    read_3mf_file_to_model(wrapper, model_path)
    mesh_obj = wrapper.GetMeshObjectByID(1)
    # doing a deformation over all vertices (subtract bounding box)
    mf_num_verts = mesh_obj.GetVertexCount()
    for i in range(mf_num_verts):
        new_vect = shift_vert_to_center(mesh_obj.GetVertex(i))
        mesh_obj.SetVertex(i, new_vect)

# Load mesh
model_name = "pi 3mm"
model_path = f'input_models/{model_name}.stl'
# make binary stl from ASCII
model_mesh = mesh.Mesh.from_file(model_path)
model_mesh.save(f'input_models/{model_name}_binary.stl')
# make ASCII stl from binary
model_mesh.save(f'input_models/{model_name}_ascii.stl', mode=stl.Mode.ASCII)
# debug statements
# read triangle mesh
if os.path.isfile(f'input_models/{model_name}.stl'):
    print("Found file!")
    model_mesh = o3d.io.read_triangle_mesh(f'input_models/{model_name}_ascii.stl')

# convert to tetrahedral mesh
input_tet = tetgen.TetGen(np.asarray(model_mesh.vertices), np.asarray(model_mesh.triangles))
# input_tet.make_manifold() # comment out if not needed
input_tet.tetrahedralize()
input_tet = input_tet.grid

surface = input_tet.extract_surface() #pyvista


In [ ]:
a = np.array([[1, 0, 5],
              [40, 40, 10],
              [2, 1, 3],
              [0, 0, 1],
              [40, 40, 10]])

b = np.array([[40, 40, 10]])

mask = (a[:, :] == b[:, None]).all(axis=-1).any(axis=0)
print(mask)
print(a[mask])
print(np.where(mask))

print("Now my code")
# mesh_verts = np.array([[40.0, 40.0, 10.0],
#                     [37.5, 40.0, 10.0],
#                     [40.0, 42.5, 10.0,]])
# mesh_verts = np.asarray(model_mesh.vertices)[0:3, :]

mesh_verts = np.asarray(model_mesh.vertices)
surf_verts = surface.points
tet_verts = input_tet.points
undeformed_tet = input_tet.copy()
undeformed_verts = undeformed_tet.points
undeformed_surf_verts = undeformed_tet.extract_surface().points

mf_num_verts = mesh_obj.GetVertexCount()

check_verts = surf_verts
check_range = mf_num_verts
correspondence_list = np.full(check_range, -1)
for i in range(check_range):
    curr_vert = mesh_obj.GetVertex(i)
    curr_vert = mf_verts_to_np_array(curr_vert)

    # print(mesh_verts)
    match_vert_mask = (np.isclose(check_verts[:, :], curr_vert[:, None])).all(axis=-1).any(axis=0) 
    # print(match_vert_mask)
    match_indices = np.where(match_vert_mask)
    # print(match_indices)
    # print(len(match_indices[0]))
    if len(match_indices[0]) > 0:
        correspondence_list[i] = (match_indices[0][0])
print(correspondence_list)
if -1 in correspondence_list:
    print("No correspondence")
else:
    print("YAY found correspondence")
mf_verts = np.asarray([[x.Coordinates[0], x.Coordinates[1], x.Coordinates[2]] for x in mesh_obj.GetVertices()])

print("check verts:\n", check_verts)
print("Curr vert:\n", curr_vert)
# print(f"First {check_range} 3mf vertices:")
# for i in range(check_range):
#     print(mf_verts[i])
# print(f"According to correspondence list, {check_range} matching vertices in dataset:")
# for i in range(check_range):
#     print(check_verts[correspondence_list[i]])
# TEST BLOCK SUCCESSFUL

[False  True False False  True]
[[40 40 10]
 [40 40 10]]
(array([1, 4]),)
Now my code
[  7   4   0   5   8   6  12  13  16  17  20  19  24  21  25  26  11   9
  10  34  33  35  38  36  39  53  54  44  45  42  46  50  49  59  57  56
  60  14  15  51  52  55   3   2   1  69  70  65  71  73  72  77  78  80
  81  79  85  87  84 136 141  96  95  88  90  92  89  99  98  97 101 104
 100 107 108 111 116 113 114 120 122 125 121  47  58  74  75  76 131 132
  86 133 134 135 140 139  93 143 142  94 146 147  91 149 148 105 150 151
 102 156 155 103 159 157 117 161 160 115 162 163 112 164 165 123 170 168
 124 171  43  48 191 189 174 175 172 180 177 176 182 183 185 187  83  82
 190 186 184  68  67  66 196 192 198 199 202 201 204 203 209 208 216 214
 178 181 193 194 200 218 173 179 230 224 229 228 231 232 234 233 236 235
 220 222 207 210 225 226 221 237 206 205 260 257 261 256 246 244 240 245
 248 247 249 250 254 253 258 259 251 262 263 255 267 268 265 271 270 269
 275 274 277 279 281 282 242 241 284 2

In [10]:
# TEST CODE BLOCK
# change first vertex in 3mf to all 1s
# working on pi 3mm, saving to a copy

# getting vertices from lib3mf
model_path = "./input_models/pi 3mm.3mf"
extension = "3mf"
output_filepath = f"../Files from WSL/pi edited.{extension}"
if os.path.isfile(model_path):
    wrapper = get_wrapper()
    wrapper = wrapper.CreateModel()
    read_3mf_file_to_model(wrapper, model_path)
    mesh_obj = wrapper.GetMeshObjectByID(1)
    # doing a deformation over all vertices (subtract bounding box)
    mf_num_verts = mesh_obj.GetVertexCount()

def make_3mf_vertex(position):
    x, y, z = position[0], position[1], position[2]
    new_vert = lib3mf.Position()
    new_vert.Coordinates[0] = x
    new_vert.Coordinates[1] = y
    new_vert.Coordinates[2] = z
    return new_vert

# write vertex id 0 as 1, 1, 1
one_vert = make_3mf_vertex(np.ones(3))
original_vert = make_3mf_vertex(np.array([40.0, 40.0, 10.0]))
mesh_obj.SetVertex(0, original_vert)

# write to file
writer = wrapper.QueryWriter(extension)
print(f"Writing {output_filepath}...")
writer.WriteToFile(output_filepath)
print("Done")
# TEST BLOCK SUCCESSFUL!

Writing ../Files from WSL/pi edited.3mf...
Done
